In [1]:
import pickle
import re
import unidecode
from collections import defaultdict

from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
with open('../data/sample_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
# replace words within paranthesis and remove non alphanumeric characters
# get_rid of `the` which can cause difference
pattern = re.compile(r'\([^)]*\)|[^a-zA-Z0-9 -]|\s|(^|[^a-zA-Z0-9])the[^a-zA-Z0-9]')
# unidecode to remove accents
normalize_text = lambda text: pattern.sub("", unidecode.unidecode(text).lower())

In [4]:
with open('../data/name_to_wiki_id.pkl', 'rb') as f:
    name_to_wiki_id = pickle.load(f)

In [5]:
with open('../data/anchor_to_wiki_id.pkl', 'rb') as f:
    anchor_to_wiki_id = pickle.load(f)

In [6]:
len(name_to_wiki_id.keys())

10898733

In [7]:
result = []
for article in tqdm(data):
    text = article[0]
    name_entities = article[1]
    candidates = {}
    for entity in name_entities:
        entity_name = entity[0]
        candidates[entity] = name_to_wiki_id.get(normalize_text(entity_name))
        if candidates[entity] is None:
            candidates[entity] = []
        anchor_list = anchor_to_wiki_id.get(entity_name)
        if anchor_list:
            candidates[entity] += anchor_list
#         candidates[entity] = list(set(candidates[entity]))
    result.append([text, candidates])

100%|██████████| 4978/4978 [00:00<00:00, 21823.16it/s]


In [8]:
with open('../data/sample_data_candidates.pkl', 'wb') as f:
    pickle.dump(result, f)